In [1]:
import pandas as pd
import tensorflow as tf

df_train_test = pd.DataFrame(columns=["Home Team", "Away Team",
                                      "Points Obtained Last Season", "Points Obtained by Opponent Last Season",
                                      "Goals Scored Last Season", "Goals Conceded by Opponent Last Season",
                                      "Points Obtained This Season", "Points Obtained by Opponent This Season",
                                      "Goals Scored This Season", "Goals Conceded by Opponent This Season",
                                      "Goals Scored in Game"])
teams = {}
teams_stats = {}
for i in range(10, 18):
    df = pd.read_csv("./archive-3/Datasets/20" + str(i) + "-" + str(i + 1) + ".csv")
    teams[i] = set(df["HomeTeam"].unique())
    teams_stats[i] = {}
    for team in teams[i]:
        teams_stats[i][team] = [0, 0, 0, 0]
    for index, row in df.iterrows():
        if i > 10 and teams_stats[i][row["HomeTeam"]][3] > 0 and teams_stats[i][row["AwayTeam"]][3]:
            row1 = [1, 0, 1, 1, 1, 1, 0, 0, 0, 0, row["FTHG"]]
            row2 = [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, row["FTAG"]]
            if row["HomeTeam"] in teams[i - 1]:
                row1[2] = teams_stats[i - 1][row["HomeTeam"]][0] / 38
                row1[4] = teams_stats[i - 1][row["HomeTeam"]][1] / 38
                row2[3] = teams_stats[i - 1][row["HomeTeam"]][0] / 38
                row2[5] = teams_stats[i - 1][row["HomeTeam"]][2] / 38
            if row["AwayTeam"] in teams[i - 1]:
                row1[3] = teams_stats[i - 1][row["AwayTeam"]][0] / 38
                row1[5] = teams_stats[i - 1][row["AwayTeam"]][2] / 38
                row2[2] = teams_stats[i - 1][row["AwayTeam"]][0] / 38
                row2[4] = teams_stats[i - 1][row["AwayTeam"]][1] / 38
            row1[6] = teams_stats[i][row["HomeTeam"]][0] / teams_stats[i][row["HomeTeam"]][3]
            row1[8] = teams_stats[i][row["HomeTeam"]][1] / teams_stats[i][row["HomeTeam"]][3]
            row2[7] = teams_stats[i][row["HomeTeam"]][0] / teams_stats[i][row["HomeTeam"]][3]
            row2[9] = teams_stats[i][row["HomeTeam"]][2] / teams_stats[i][row["HomeTeam"]][3]
            row1[7] = teams_stats[i][row["AwayTeam"]][0] / teams_stats[i][row["AwayTeam"]][3]
            row1[9] = teams_stats[i][row["AwayTeam"]][2] / teams_stats[i][row["AwayTeam"]][3]
            row2[6] = teams_stats[i][row["AwayTeam"]][0] / teams_stats[i][row["AwayTeam"]][3]
            row2[8] = teams_stats[i][row["AwayTeam"]][1] / teams_stats[i][row["AwayTeam"]][3]
            df_train_test.loc[-1] = row1
            df_train_test.index = df_train_test.index + 1
            df_train_test = df_train_test.sort_index()
            df_train_test.loc[-1] = row2
            df_train_test.index = df_train_test.index + 1
            df_train_test = df_train_test.sort_index()
        if i < 18:
            teams_stats[i][row["HomeTeam"]][3] += 1
            teams_stats[i][row["AwayTeam"]][3] += 1
            teams_stats[i][row["HomeTeam"]][1] += row["FTHG"]
            teams_stats[i][row["HomeTeam"]][2] += row["FTAG"]
            teams_stats[i][row["AwayTeam"]][1] += row["FTAG"]
            teams_stats[i][row["AwayTeam"]][2] += row["FTHG"]
            if row["FTHG"] > row["FTAG"]:
                teams_stats[i][row["HomeTeam"]][0] += 3
            elif row["FTHG"] == row["FTAG"]:
                teams_stats[i][row["HomeTeam"]][0] += 1
                teams_stats[i][row["AwayTeam"]][0] += 1
            else:
                teams_stats[i][row["AwayTeam"]][0] += 3

/Users/muyuanyang/opt/anaconda3/envs/ml/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df_train = df_train_test[:-760]

In [ ]:
from tensorflow.keras import layers, models, regularizers, optimizers
from tensorflow.keras.optimizers.schedules import ExponentialDecay
"""
model = models.Sequential()
model.add(layers.Dense(32, input_dim=10, activation=tf.nn.leaky_relu, kernel_regularizer=regularizers.l2(l=0.1)))
#model.add(layers.Dropout(0.1))
model.add(layers.Dense(32, activation=tf.nn.leaky_relu, kernel_regularizer=regularizers.l2(l=0.1)))
model.add(layers.Dense(1, activation='linear'))

initial_learning_rate = 0.01
lr_schedule = optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True)
optimizer = optimizers.Adam(learning_rate=lr_schedule)

model.compile(loss='mean_absolute_error', optimizer=optimizer, metrics=['mean_squared_error'])
"""
model = models.Sequential()
model.add(layers.Dense(32, input_dim=10, activation=tf.nn.leaky_relu))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(32, activation=tf.nn.leaky_relu))
model.add(layers.Dense(1, activation='linear'))

model.compile(loss='mean_absolute_error', optimizer="adam", metrics=['mean_squared_error'])

In [66]:
model.fit(df_train[["Home Team", "Away Team",
                    "Points Obtained Last Season", "Points Obtained by Opponent Last Season",
                    "Goals Scored Last Season", "Goals Conceded by Opponent Last Season",
                    "Points Obtained This Season", "Points Obtained by Opponent This Season",
                    "Goals Scored This Season", "Goals Conceded by Opponent This Season"]],
          df_train[["Goals Scored in Game"]], epochs=75, validation_split=0.2)

Epoch 1/75
  1/111 [..............................] - ETA: 37s - loss: 2.2188 - mean_squared_error: 7.8297

2024-02-10 17:01:57.952176: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


111/111 [==============================] - ETA: 0s - loss: 1.0307 - mean_squared_error: 1.8287

2024-02-10 17:01:58.957654: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


111/111 [==============================] - 1s 9ms/step - loss: 1.0307 - mean_squared_error: 1.8287 - val_loss: 0.9069 - val_mean_squared_error: 1.4708
Epoch 2/75
111/111 [==============================] - 1s 6ms/step - loss: 0.9495 - mean_squared_error: 1.5452 - val_loss: 0.9130 - val_mean_squared_error: 1.3843
Epoch 3/75
111/111 [==============================] - 1s 6ms/step - loss: 0.9401 - mean_squared_error: 1.4935 - val_loss: 0.9041 - val_mean_squared_error: 1.4328
Epoch 4/75
111/111 [==============================] - 1s 6ms/step - loss: 0.9324 - mean_squared_error: 1.4647 - val_loss: 0.9075 - val_mean_squared_error: 1.4314
Epoch 5/75
111/111 [==============================] - 1s 6ms/step - loss: 0.9299 - mean_squared_error: 1.4512 - val_loss: 0.9041 - val_mean_squared_error: 1.4237
Epoch 6/75
111/111 [==============================] - 1s 6ms/step - loss: 0.9280 - mean_squared_error: 1.4374 - val_loss: 0.8995 - val_mean_squared_error: 1.3569
Epoch 7/75
111/111 [===================

In [67]:
df_test = df_train_test[-760:]
preds = model.predict(df_test[["Home Team", "Away Team",
                               "Points Obtained Last Season", "Points Obtained by Opponent Last Season",
                               "Goals Scored Last Season", "Goals Conceded by Opponent Last Season",
                               "Points Obtained This Season", "Points Obtained by Opponent This Season",
                               "Goals Scored This Season", "Goals Conceded by Opponent This Season"]])

24/24 [==============================] - 0s 3ms/step


2024-02-10 17:03:02.001179: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [68]:
print(preds)

[[0.33008498]
 [2.472285  ]
 [1.017069  ]
 [1.0722996 ]
 [2.1453795 ]
 [0.6727581 ]
 [1.1734298 ]
 [0.9643942 ]
 [0.7663325 ]
 [2.6988437 ]
 [0.8089261 ]
 [1.1194159 ]
 [0.4639475 ]
 [0.6714797 ]
 [1.897802  ]
 [1.0726577 ]
 [0.9275317 ]
 [1.7212205 ]
 [1.281653  ]
 [0.67118424]
 [0.3043872 ]
 [2.238621  ]
 [0.96519005]
 [0.925365  ]
 [1.5585083 ]
 [0.9800693 ]
 [0.9280621 ]
 [1.6607462 ]
 [1.0175991 ]
 [0.9837698 ]
 [2.0540848 ]
 [0.947078  ]
 [0.95386934]
 [1.0044582 ]
 [0.9494742 ]
 [1.0618879 ]
 [0.42288992]
 [2.255828  ]
 [0.9876518 ]
 [1.0624121 ]
 [0.8682312 ]
 [2.0187497 ]
 [1.1649265 ]
 [0.9937495 ]
 [0.95338655]
 [1.0630782 ]
 [1.0325211 ]
 [0.973987  ]
 [0.94762707]
 [0.97974944]
 [1.5723569 ]
 [0.9792088 ]
 [0.48172042]
 [2.4829571 ]
 [0.9409299 ]
 [1.2564275 ]
 [0.9699973 ]
 [1.0843872 ]
 [1.341529  ]
 [0.9646789 ]
 [0.90966654]
 [2.0748804 ]
 [0.9271263 ]
 [1.7542468 ]
 [1.2324004 ]
 [0.9750979 ]
 [0.9722842 ]
 [0.97401106]
 [0.935959  ]
 [1.2788912 ]
 [1.1921872 ]
 [1.38

In [69]:
model.evaluate(df_test[["Home Team", "Away Team",
                               "Points Obtained Last Season", "Points Obtained by Opponent Last Season",
                               "Goals Scored Last Season", "Goals Conceded by Opponent Last Season",
                               "Points Obtained This Season", "Points Obtained by Opponent This Season",
                               "Goals Scored This Season", "Goals Conceded by Opponent This Season"]],
              df_test[["Goals Scored in Game"]])

24/24 [==============================] - 0s 5ms/step - loss: 0.9115 - mean_squared_error: 1.4914


[0.9114514589309692, 1.4914499521255493]

In [70]:
model.save('./prediction_model')

INFO:tensorflow:Assets written to: ./prediction_model/assets
